In [21]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [22]:
data=pd.read_csv('avocado.csv')

In [23]:
data.shape

(18249, 14)

In [24]:
data=data.sort_values(by='Date',ascending='True')

In [25]:
data.loc[data['region']=='Houston'].head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
10113,51,2015-01-04,1.22,8938.32,7009.77,671.88,0.00,1256.67,1256.67,0.00,0.0,organic,2015,Houston
987,51,2015-01-04,0.71,1062990.62,506426.58,436347.57,4378.92,115837.55,90299.85,25537.70,0.0,conventional,2015,Houston
986,50,2015-01-11,0.78,1062071.65,463272.52,441785.80,4094.47,152918.86,126995.76,25923.10,0.0,conventional,2015,Houston
10112,50,2015-01-11,1.26,7561.00,6142.35,818.65,0.00,600.00,600.00,0.00,0.0,organic,2015,Houston
985,49,2015-01-18,0.77,1017854.16,458532.19,429687.47,9353.74,120280.76,107320.39,12960.37,0.0,conventional,2015,Houston


In [26]:
data=data[['Date','AveragePrice','Total Volume','year','region','type']]

In [27]:
filt=(data['region']=='Houston') & (data['type']=='conventional')
data=data.loc[filt].drop(columns=['region','type'])

In [28]:
data=data.sort_values(by='Date',ascending='True')

In [29]:
data['Date'] = pd.to_datetime(data['Date'])

<h1> Average price and total volume(sales count) is weekly average</h1>

In [30]:
data.set_index('Date',inplace=True)

<h1> Doing log-log transformation </h1>

In [31]:
data['log_AveragePrice'] = np.log(data['AveragePrice'])
data['log_Total Volume'] = np.log(data['Total Volume'])
data['Total Revenue']=data['AveragePrice']*data['Total Volume']
data.drop(columns=['year'])

,AveragePrice,Total Volume,log_AveragePrice,log_Total Volume,Total Revenue
Date,,,,,
2015-01-04,0.71,1062990.62,-0.342490,13.876597,7.547233e+05
2015-01-11,0.78,1062071.65,-0.248461,13.875732,8.284159e+05
2015-01-18,0.77,1017854.16,-0.261365,13.833207,7.837477e+05
2015-01-25,0.77,983910.94,-0.261365,13.799291,7.576114e+05
2015-02-01,0.72,1280364.01,-0.328504,14.062655,9.218621e+05
...,...,...,...,...,...
2018-02-25,0.67,1657524.28,-0.400478,14.320836,1.110541e+06
2018-03-04,0.71,1501699.41,-0.342490,14.222108,1.066207e+06
2018-03-11,0.83,1324475.69,-0.186330,14.096527,1.099315e+06


In [32]:
X=np.array(data['log_AveragePrice'])
y=np.array(data['log_Total Volume'])
sc_X=StandardScaler()
sc_y=StandardScaler()
X=sc_X.fit_transform(X.reshape(-1, 1))
y=sc_y.fit_transform(y.reshape(-1, 1))

<h2> Null Hypothesis: Price and Demand are not related.</h2>

In [33]:
model=sm.OLS(y,X).fit()
print(model.summary())


                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.371
Model:                            OLS   Adj. R-squared (uncentered):              0.367
Method:                 Least Squares   F-statistic:                              98.98
Date:                Mon, 04 Oct 2021   Prob (F-statistic):                    1.27e-18
Time:                        22:11:58   Log-Likelihood:                         -200.66
No. Observations:                 169   AIC:                                      403.3
Df Residuals:                     168   BIC:                                      406.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [34]:
model.pvalues

array([1.26523298e-18])

<h2> The small P values indicate that we can reject the null hypothesis </h2>